In [ ]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


In [364]:
# Todo 
# - rejectReason dans le match batch
# - checker Photon


In [ ]:
import urllib3

In [ ]:
http = urllib3.PoolManager()

# Functions

In [ ]:
ws_hostname = "127.0.1.1"
ws_hostname = "172.27.0.64"


street_field  = "streetName"
housenbr_field = "houseNumber"
postcode_field = "postCode"
city_field  =    "city"
country_field =  "country"
addr_key_field = "addrKey"

# ws_hostname = "192.168.1.3"

In [ ]:

sample_size = None# 1000
def get_addresses(addresses_filename):
    addresses = pd.read_csv(addresses_filename,  
                            usecols = lambda x: x in [addr_key_field, "addr_key",
                                       country_field, 
                                       postcode_field, 
                                       city_field, 
                                       street_field, 
                                       housenbr_field,
                                       country_field], 
                           dtype={postcode_field: str, housenbr_field: str, country_field: str})
    
    
    addresses = addresses.rename(columns={"addr_key": addr_key_field})
    if country_field in addresses: 
        addresses[country_field] =addresses[country_field].fillna("Belgique")
    else: 
        addresses[country_field] = "Belgique"
    #addresses = addresses.rename(columns={"index":addr_key_field})
    
    addresses = addresses[addresses[street_field].notnull() & addresses[city_field].notnull()] # & addresses[country_field].isnull() ]
    
    addresses[postcode_field] = addresses[postcode_field].astype(str)
    
    if sample_size: 
        addresses = addresses.sample(sample_size, random_state=0)
    
    return addresses#.drop(country_field, axis=1)

In [508]:
def call_ws(addr_data, check_result=True, structured_osm=False, with_rejected=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    try: 
        r = http.request(
        'GET',
        f'http://{ws_hostname}:5000//REST/nominatimWrapper/v0.1/search',
        fields= { 
             street_field: addr_data[street_field],
             housenbr_field: addr_data[housenbr_field],
             city_field: addr_data[city_field],
             postcode_field: addr_data[postcode_field],
             country_field: addr_data[country_field],
             "checkResult" : "yes" if check_result else "no",
             "structOsm" : "yes" if structured_osm else "no",
            "withRejected": "yes" if with_rejected else "no"

        })
        

    except Exception as e:
        print("Exception !")
        print(addr_data)
        print(e)
        raise e
        
    if r.status == 204:
        print("No result!")
        print(addr_data)
        print(r.data)
        return
    elif r.status == 200:
        try:
            res = json.loads(r.data.decode('utf-8'))
            res["time"] = (datetime.now() - t).total_seconds()
        except ValueError as ve:

            print("Cannot decode result:")
            print(ve)
            print(r.data.decode('utf-8'))
            return r.data
        return res
    else: 
        print(f"Unknown return code: {r.status} ")
        print(r.data)



In [354]:
def split_columns(addresses_batch):
    output = pd.DataFrame(index=addresses_batch.index, dtype=str)

    for f in addresses_batch:
        print(f)
        grp= addresses_batch[f].apply(pd.Series, dtype=str)
        grp = pd.concat({f: grp}, names=['L0', 'L1'], axis=1)
        output = pd.concat([output, grp], axis=1)
    output.columns = pd.MultiIndex.from_tuples(output.columns)
    return output  

In [416]:
def call_ws_batch(addr_data, mode="geo", with_rejected=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    file_data = addr_data.to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/REST/nominatimWrapper/v0.1/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "withRejected": "yes" if with_rejected else "no",
        "checkResult" : "yes" if check_result else "no",
        "structOsm"   : "yes" if structured_osm else "no",
        #"extra_house_nbr": "no"
    })
    
    try:
        
        res_json = json.loads(r.data.decode('utf-8'))
        if "match" in res_json:
            match =  split_columns(pd.DataFrame(res_json["match"]))
            rejected= split_columns(pd.DataFrame(res_json["rejected"])) if len(res_json["rejected"])>0 else pd.DataFrame()
            
        else :
            match =  split_columns(pd.DataFrame(res_json))
            rejected=None
    except ValueError as ve:
        
        print("Cannot decode result:")
        print(ve)
        print(r.data.decode('utf-8'))
     
        return r.data
    
    if rejected is not None:
        return match, rejected
    
    else :
        return match
    


In [ ]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d:  "NONE" if d is None else "error" if "error" in d else "match" if "match" in d else "no_result")
    addresses["time"]  = addresses.json.apply(lambda d: "NONE" if d is None else d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: "NONE" if d is None else d["timing"] if "timing" in d else {})
    
    # for f in ["method", "placeRank"]:
    #     addresses[f]= addresses.json.apply(lambda d: d["match"][0][f] if d is not None and len(d)>0 and "match" in d else "none")
    
    for field in [street_field, housenbr_field, postcode_field, city_field]:
        addresses["addrOut"+field]= addresses.json.apply(lambda d: d["match"][0]["address"][field] if d is not None and len(d)>0 and "match" in d and field in d["match"][0]["address"] else "")
    return 

In [ ]:
# def expand_batch_address(addresses):
#     for field in [street_field, housenbr_field, postcode_field, city_field]:
#         addresses["addrOut"+field]= addresses.address.apply(lambda d: d[field] if d is not None and field in d else "")
#     return 

# Calls

## Single address calls

In [519]:
res=call_ws({street_field:   "Avenue Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgique"}, check_result=True, structured_osm=False, with_rejected=True)
res

{'match': [{'work': {'transformedAddress': 'Avenue Fonsny, 20, 1060 Saint-Gilles, Belgique',
    'osmOrder': 0,
    'method': 'orig'},
   'input': {'addrKey': '1',
    'streetName': 'Avenue Fonsny',
    'postCode': '1060',
    'houseNumber': '20',
    'city': 'Saint-Gilles',
    'country': 'Belgique'},
   'nominatim': {'placeId': 182128,
    'lat': '50.8358677',
    'lon': '4.3385087',
    'displayName': '20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
    'placeRank': 30,
    'category': 'place',
    'type': 'house',
    'road': 'Avenue Fonsny - Fonsnylaan',
    'countryCode': 'be',
    'country': 'België / Belgique / Belgien',
    'region': 'Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest',
    'town': 'Saint-Gilles - Sint-Gillis',
    'postcode': '1060',
    'county': 'Brussel-Hoofdstad - Bruxelles-Capitale',
    'houseN

In [521]:
res=call_ws({street_field:   "Avenue Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgique"}, check_result=False, structured_osm=False, with_rejected=True)
res

{'match': [{'work': {'method': 'fast',
    'transformedAddress': 'Avenue Fonsny, 20, 1060 Saint-Gilles, Belgique'},
   'nominatim': {'displayName': '20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
    'placeId': 182128,
    'lat': '50.8358677',
    'lon': '4.3385087',
    'placeRank': 30,
    'houseNumber': '20',
    'road': 'Avenue Fonsny - Fonsnylaan',
    'town': 'Saint-Gilles - Sint-Gillis',
    'county': 'Brussel-Hoofdstad - Bruxelles-Capitale',
    'region': 'Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest',
    'postcode': '1060',
    'country': 'België / Belgique / Belgien',
    'countryCode': 'be'},
   'output': {'streetName': 'Avenue Fonsny - Fonsnylaan',
    'city': 'Saint-Gilles - Sint-Gillis',
    'houseNumber': '20',
    'country': 'België / Belgique / Belgien',
    'postCode': '1060',
    'inHouseNbr': '',
  

## Batch calls (row by row)

In [620]:
addresses = pd.concat([
   get_addresses("data/address_restobe.csv.gz"),
   get_addresses("data/address_best.csv.gz"),
    get_addresses("data/address_cbe.csv.gz")])
addresses = addresses.reset_index(drop=True)
addresses[addr_key_field] = addresses.index.astype(str)
addresses

,addrKey,streetName,houseNumber,city,postCode,country
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique
1,1,Rue de la Halle,11,NAMUR,5000,Belgique
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique
3,3,rue du Centre,200,WAIMES,4950,Belgique
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique
...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique


In [601]:
# addresses = addresses[addresses.addr_key.isin([ "2078829"])]#"1622",
# addresses.dtypes


### Simple way

In [621]:
addresses = addresses[addresses.streetName.str.lower().str.contains("gustave desmetstraat")]
# addresses = addresses.loc[[100,112,118,142,155,179,198]]
# addresses = addresses.loc[[ 94,  100,  198, ]]
# addresses = addresses.loc[100:200]
addresses

,addrKey,streetName,houseNumber,city,postCode,country
1460,1460,Gustave Desmetstraat,15,Zwevegem,8550,Belgique


In [624]:
addresses_seq = addresses.copy()

t = datetime.now()
addresses_seq["json"] = addresses_seq.fillna("").progress_apply(call_ws, check_result=True, structured_osm=True, axis=1)
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_seq.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 128.78 seconds, 7.77 it/s
# Fastmode:     68.80 seconds, 14.54 it/s

#Resto dataset: 
# Normal mode: 145.73 seconds, 6.86 it/s
# Fast mode:    82.99 seconds, 12.05 it/s

# Best dataset:
# Normal mode: 108.53 seconds, 9.21 it/s
# Fast mode: 37.44 seconds, 26.71 it/s

  0%|          | 0/1 [00:00<?, ?it/s]

2.39 seconds, 0.42 it/s


In [607]:
addresses_seq[addresses_seq.json.isnull()].index

Int64Index([  94,  100,  198,  254,  335,  432,  443,  473,  784,  905,  906,
             917, 2163, 2206, 2556, 2715, 2882, 2977],
           dtype='int64')

In [574]:
addresses_seq["method"] = addresses_seq.json.apply(lambda js: js["match"][0]["work"]["method"] if js is not None else None)

In [575]:
addresses_seq["method"].value_counts()

fast    86
Name: method, dtype: int64

In [550]:
expand_json(addresses_seq)
addresses_seq

KeyError: 'method'

In [556]:
x = split_columns(addresses_seq[["json"]])

json


json          
                                                 match      time
117  [{'work': {'transformedAddress': 'Chaussee de ...  0.227614
118  [{'work': {'transformedAddress': 'rue masuy, 8...   0.48119
119  [{'work': {'transformedAddress': 'Avenue du Pr...  0.215907
120  [{'work': {'transformedAddress': 'Place eugene...  0.235003

In [ ]:
addresses_seq.method.value_counts()

### Using Dask

In [ ]:
addresses_dask = addresses.copy()

In [ ]:
addresses_dask

In [ ]:
t = datetime.now()
dd_addresses = dd.from_pandas(addresses_dask.fillna(""), npartitions=8)

dask_task = dd_addresses.apply(call_ws, check_result=False, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses_dask["json"] = dask_task.compute()
    
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_dask.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 24.52 seconds, 40.79 it/s
# Fastmode:    15.81 seconds, 63.27 it/s


# Resto dataset:
# Normal mode: 27.86 seconds, 35.89 it/s
# Fast mode:   18.44 seconds, 54.23 it/s

# Best dataset: 
# Normal mode: 16.11 seconds, 62.07 it/s
# Fast mode:    9.76 seconds, 102.42 it/s

In [ ]:
# 1000, 1 worker: 4m18
# 4 workers, npart=4 : 1m20
# 8 workers, npart=4 : 1m20
# 8 workers, npart=8 : 44s

# with checker=False:
# 8 workers, npart=8 : 24s


In [ ]:
expand_json(addresses_dask)
addresses_dask

In [ ]:
addresses_dask.method.value_counts()#.json.loc[550]

In [ ]:
mg = addresses_seq[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].merge(
    addresses_dask[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]], how="outer", indicator=True)
if mg.shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

In [ ]:
mg[mg._merge != "both"].sort_values("addrKey")


## Batch calls (batch WS)

### Single block

In [454]:
# addresses=addresses[addresses.addrKey.isin(["1075", "1331", "2616"])]
addresses=addresses[addresses.addrKey.isin(["2206", "2556", "2569", "2715"])]
# addresses=addresses.iloc[0:10]
addresses

,addrKey,streetName,houseNumber,city,postCode,country
2206,2206,105 A.H.JASPAR,NaN,Saint-Gilles,1060,Belgique
2556,2556,17 A.H.HOLLEVOET,NaN,Molenbeek-Saint-Jean,1080,Belgique
2569,2569,"Zone d'activités Sud, Bail.",6,Somme-Leuze,5377,Belgique
2715,2715,Fiestaux,NaN,Charleroi,6010,Belgique


In [474]:
t = datetime.now()

addresses_batch, rejected = call_ws_batch(addresses[[addr_key_field, 
                                           street_field, housenbr_field, postcode_field, city_field, country_field]], 
                                mode="long", 
                                check_result=True, 
                                structured_osm=True,
                                with_rejected=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO dataset: 33.94 seconds, 29.46 it/s
# Best:        24.99 seconds, 40.01 it/s
# Resto:       38.33 seconds, 26.09 it/s

addresses_batch

work
input
nominatim
output
check
work
input
nominatim
output
check
107.16 seconds, 28.00 it/s


work                     \
                                     transformedAddress osmOrder    method   
0          Chaussée de Namur, 198, 5310 LEUZE, Belgique      0.0      orig   
1             Rue de la Halle, 11, 5000 NAMUR, Belgique      0.0      orig   
2                Steenweg, 269, 9810 Nazareth, Belgique      0.0      orig   
3              Centrumstraat, 5, 3012 Louvain, Belgique      0.0      orig   
4     Rue Félix De Cuyper, 39, 1070 Anderlecht, Belg...      0.0      orig   
...                                                 ...      ...       ...   
2995                         2070 ZWIJNDRECHT, Belgique      0.0  nostreet   
2996                             4950 MALMEDY, Belgique      0.0  nostreet   
2997                           5580 ROCHEFORT, Belgique      0.0  nostreet   
2998                        4845 SART-LEZ-SPA, Belgique      0.0  nostreet   
2999                   5537 ANNEVOIE-ROUILLON, Belgique      0.0  nostreet   

                  input                                                     \
     retryOn_26 addrKey                                         streetName   
0           NaN       0                                  Chaussée de Namur   
1           NaN       1                                    Rue de la Halle   
2           NaN    1000                                           Steenweg   
3           NaN    1001                                      Centrumstraat   
4           NaN    1002                                Rue Félix De Cuyper   
...         ...     ...                                                ...   
2995        NaN     669                                     Stationsstraat   
2996        NaN     682                          Rue du Lac | Robertville,   
2997        NaN     758  Rue du Sourd d'Ave | situé au confins de la N9...   
2998        NaN     784                                  route de Balmoral   
2999        NaN      93                                  route des Jardins   

                                                        ...   output  \
     houseNumber postCode               city   country  ... postCode   
0            198     5310              LEUZE  Belgique  ...     5310   
1             11     5000              NAMUR  Belgique  ...     5000   
2            269     9810           Nazareth  Belgique  ...     9810   
3              5     3012            Louvain  Belgique  ...     3012   
4             39     1070         Anderlecht  Belgique  ...     1070   
...          ...      ...                ...       ...  ...      ...   
2995          93     2070        ZWIJNDRECHT  Belgique  ...     2070   
2996          24     4950            MALMEDY  Belgique  ...     4960   
2997           5     5580          ROCHEFORT  Belgique  ...     5580   
2998          35     4845       SART-LEZ-SPA  Belgique  ...     4845   
2999          35     5537  ANNEVOIE-ROUILLON  Belgique  ...     5537   

                                                                    \
                                                  other inHouseNbr   
0                                       Namur - Éghezée        198   
1                                                   NaN         11   
2                                Gent - Oost-Vlaanderen        269   
3                               Leuven - Vlaams-Brabant          5   
4     Brussel-Hoofdstad - Bruxelles-Capitale - Aa - ...         39   
...                                                 ...        ...   
2995                                          Antwerpen         93   
2996                                   Liège - Verviers         24   
2997                                     Namur - Dinant          5   
2998                                   Liège - Verviers         35   
2999                                     Namur - Dinant         35   

                                 check                                    \
     lpostHouseNbr lpostUnit SIMStreet SIMStreetWhich SIMHouseNbr SIMZip   
0              198   

In [466]:
addresses_batch[("work", "method")].value_counts()

photon    4
Name: (work, method), dtype: int64

In [467]:
rejected[("work", "rejectReason")].value_counts()

tail        3
mismatch    1
Name: (work, rejectReason), dtype: int64

In [468]:
rejected[rejected[("work", "rejectReason")] == "mismatch"]#.iloc[5]

work                                input  \
                   transformedAddress osmOrder rejectReason method addrKey   
0  Fiestaux, 6010 Charleroi, Belgique        0     mismatch   orig    2715   

                                           nominatim  ...    output  \
  streetName postCode       city   country   placeId  ...      city   
0   Fiestaux     6010  Charleroi  Belgique   2027171  ...  Couillet   

                                                     \
  houseNumber                      country postCode   
0           6  België / Belgique / Belgien     6010   

                                                         check                 \
                                               other SIMStreet SIMStreetWhich   
0  Charleroi - Hainaut - Crèche communale Les Fie...       0.0    namedetails   

                                          
  SIMHouseNbr SIMZip             SIMCity  
0         0.0    1.0  0.2222222222222222  

[1 rows x 39 columns]

In [494]:
addresses_batch[addresses_batch[("work", "method")] == "photon"].iloc[0] 

work       transformedAddress    Rue du Pont, 51, 5530 Yvoir, België / Belgique...
           osmOrder                                                            0.0
           method                                                           photon
           retryOn_26                                                          NaN
input      addrKey                                                            2077
           streetName                                          Rue du Pont,Godinne
           houseNumber                                                          51
           postCode                                                           5530
           city                                                              Yvoir
           country                                                        Belgique
nominatim  placeId                                                         2421051
           lat                                                          50.3486795
    

In [400]:
rejected

work                   \
                                    transformedAddress osmOrder  method   
0    Rue Coëmont, Arville, 4, 6870 Saint-Hubert, Be...        0    orig   
1               Porreyestraat, 9, 8908 Ieper, Belgique        0    orig   
2                   Fiestaux, 6010 Charleroi, Belgique        0    orig   
3           Avenue Louise, 523, 1050 Ixelles, Belgique        0    orig   
4           Avenue Louise, 523, 1050 Ixelles, Belgique        1    orig   
..                                                 ...      ...     ...   
376  Avenue Henri Jaspar - Henri Jasparlaan, 1060 S...        1  photon   
377  Avenue Henri Jaspar - Henri Jasparlaan, 1060 S...        2  photon   
378  Avenue Henri Jaspar - Henri Jasparlaan, 1060 S...        3  photon   
379  Grand-Rechain, 22, 4650 Herve, België / Belgiq...        2  photon   
380  Grand-Rechain, 22, 4650 Herve, België / Belgiq...        0  photon   

      input                                                               \
    addrKey                              streetName postCode houseNumber   
0       163                    Rue Coëmont, Arville     6870           4   
1       468                           Porreyestraat     8908           9   
2       715                                Fiestaux     6010         NaN   
3       745                           Avenue Louise     1050         523   
4       745                           Avenue Louise     1050         523   
..      ...                                     ...      ...         ...   
376     206  Avenue Henri Jaspar - Henri Jasparlaan     1060         NaN   
377     206  Avenue Henri Jaspar - Henri Jasparlaan     1060         NaN   
378     206  Avenue Henri Jaspar - Henri Jasparlaan     1060         NaN   
379     882                           Grand-Rechain     4650          22   
380     882                           Grand-Rechain     4650          22   

                                                             nominatim  ...  \
                           city                      country   placeId  ...   
0                  Saint-Hubert                     Belgique    181906  ...   
1                         Ieper                     Belgique    488431  ...   
2                     Charleroi                     Belgique   2027171  ...   
3                       Ixelles                     Belgique   3585797  ...   
4                       Ixelles                     Belgique    333227  ...   
..                          ...                          ...       ...  ...   
376  Saint-Gilles - Sint-Gillis  België / Belgique / Belgien    345033  ...   
377  Saint-Gilles - Sint-Gillis  België / Belgique / Belgien   3663517  ...   
378  Saint-Gilles - Sint-Gillis  België / Belgique / Belgien    342017  ...   
379                       Herve  België / Belgique / Belgien   2800435  ...   
380                       Herve  België / Belgique / Belgien    101538  ...   

      output                                                                  \
    postCode                                              other rejectReason   
0       6870       Neufchâteau - Luxembourg - Champs de Coëmont     mismatch   
1       8908                      Vlamertinge - West-Vlaanderen     mismatch   
2       6010  Crèche communale Les Fiestaux - Charleroi - Ha...     mismatch   
3       1050             Brussel-Hoofdstad - Bruxelles-Capitale     mismatch   
4       1050             Brussel-Hoofdstad - Bruxelles-Capitale     mismatch   
..       ...                                                ...          ...   
376     1060             Brussel-Hoofdstad - Bruxelles-Capitale         tail   
377     1060             Brussel-Hoofdstad - Bruxelles-Capitale         tail   
378     1060             Brussel-Hoofdstad - Bruxelles-Capitale         tail   
379     4650                              Liège - Grand-Rechain         tail   
380     4650                   Liège - Grand-Rechain - Verviers         tail   

                

In [346]:
output["output"]


,streetName,city,houseNumber,country,postCode,other,inHouseNbr,lpostHouseNbr,lpostUnit
0,Kriekenlaan,Zandhoven,22,België / Belgique / Belgien,2240,Antwerpen,22,22,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Poststraat,Issegem,NaN,België / Belgique / Belgien,9860,Oosterzele - Gent - Oost-Vlaanderen,19,19,NaN
3,Sylvain Dupuisstraat,Knokke-Heist,24;26;28,België / Belgique / Belgien,8300,Brugge - Knokke - Albertstrand - West-Vlaanderen,26,26,NaN
4,Oude Baan,Maasmechelen,NaN,België / Belgique / Belgien,3630,Limburg - Cité van Mechelen aan de Maas - Tong...,338,NaN,m 338
5,Kapeldreef,Heverlee,20,België / Belgique / Belgien,3001,Vlaams-Brabant - Leuven,20,20,NaN
6,Krijgslaan,Gent,NaN,België / Belgique / Belgien,9000,Oost-Vlaanderen - Sint-Pieters-Aalst,74,74,NaN
7,Rue Charles Ramaekers - Charles Ramaekersstraat,Laeken - Laken,17,België / Belgique / Belgien,1020,Brussel-Hoofdstad - Bruxelles-Capitale - Ville...,17,17,NaN
8,Fraeysstraat,Torhout,NaN,België / Belgique / Belgien,8820,Brugge - Goede Herder - West-Vlaanderen,1,1,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
addresses_batch.placeRank.fillna("none").value_counts()

In [ ]:
expand_batch_address(addresses_batch)

In [ ]:
addresses_batch.sort_values("addrKey", key=lambda x: x.astype(int))

In [ ]:
# addresses_batch[addresses_batch.method.str.contains("error")]

In [ ]:
rejected_addresses = addresses_batch[addresses_batch.reject.apply(lambda lst: len(lst)>0)].reject.apply(pd.Series).unstack().dropna().apply(pd.Series)
rejected_addresses

In [ ]:
mg = addresses_seq[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna("").replace("fast", "orig").merge(
    addresses_batch[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna(""), how="outer", indicator=True)

# mg = addresses_seq[[ "city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna("").replace("fast", "orig").merge(
#      addresses_batch[["city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

In [ ]:
mg[mg._merge != "both"]

In [ ]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="geo", check_result=False, with_rejected=True) 

In [ ]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="short", check_result=False) 

In [ ]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_rejected=True)

### Batch blocs

In [ ]:
# addresses = addresses.sample(10000, replace=True)
# addresses = addresses.reset_index(drop=True)
# addresses["addr_key"]= addresses.index.astype(str)

In [ ]:
t = datetime.now()

nb_threads=8

chunks = np.array_split(addresses, nb_threads) # addresses.shape[0]//100)

print(f"{len(chunks)} chunks on {nb_threads} threads")

import multiprocess as mp

p = mp.Pool(nb_threads)

def f(chunk):
    return call_ws_batch(chunk, mode="long", 
                        check_result=False, 
                        structured_osm=False)

with p:
     res= list(tqdm(p.imap(f, chunks), total=len(chunks)))
    
addresses_batch2 = pd.concat(res).reset_index(drop=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO:    9.28 seconds, 107.72 it/s
# Best:   6.88 seconds, 145.43 it/s
# Resto: 11.79 seconds,  84.85 it/s

In [ ]:
# addresses_batch2
expand_batch_address(addresses_batch2)

In [ ]:
mg = addresses_seq[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna("").replace("fast", "orig").merge(
     addresses_batch2[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna(""), how="outer", indicator=True)

# mg = addresses_seq[[  "city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna("").replace("fast", "orig").merge(
#     addresses_batch2[["city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

In [ ]:
mg[mg._merge != "both"].sort_values("addrKey")

## Comparing options

In [ ]:
# addresses = get_addresses("address.csv.gz")
# addresses = addresses[addresses[country_field] == "Belgique"]
# # addresses = addresses.sample(10000).copy()

In [ ]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        expand_batch_address(results[(check_label, struct_label)])
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

In [ ]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addrKey"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

In [ ]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addrKey", "placeId"]].merge(r2[["addrKey", "placeId"]], on="addrKey", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.placeId_x != mg.placeId_y][["addrKey"]]
            mismatches = mismatches.merge(addresses)
            mismatches = mismatches.merge(r1[[addr_key_field, "placeId", "method", f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}", "lpostHouseNbr"]], on=addr_key_field)
            mismatches = mismatches.merge(r2[[addr_key_field, "placeId", "method", f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}", "lpostHouseNbr"]], on=addr_key_field)
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*7 + [f"y:{k2}"]*7, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop(f"addrOut{housenbr_field}", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop(f"addrOut{housenbr_field}", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1[addr_key_field].isin(mg[mg._merge=="left_only"][addr_key_field])].merge(addresses))
            
            print(f"Only in {k2}")
            display(r2[r2[addr_key_field].isin(mg[mg._merge=="right_only"][addr_key_field])].merge(addresses))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

In [ ]:
# r1=results[('check', 'unstruct')].merge(addresses)
# r2=results[('check', 'struct')].merge(addresses)
r1=results[('nocheck', 'struct')].merge(addresses)
r2=results[('check', 'struct')].merge(addresses)
k = 2969
display(r1.query(f"addrKey=='{k}'"))
display(r2.query(f"addrKey=='{k}'"))

In [ ]:
# addresses.iloc[0:60]#.dtypes

In [ ]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

# tests

In [419]:
osm_host ="172.27.0.64:8080"
def get_osm(addr, accept_language = ""): #lg = "en,fr,nl"
    params = urllib.parse.urlencode({"q": addr,
                                    "format":"jsonv2",
                                    "accept-language":accept_language,
                                    "addressdetails":"1",
                                    "namedetails" : "1",
                                    "limit": "50"
                                    })
    
    url = "http://%s/search.php?%s"%(osm_host, params)
#     print(f"Call to OSM: {url}")
    try: 
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
            return res
            return [ {field: item[field] for field in ["place_id", "lat", "lon", "display_name", "address", "namedetails", "place_rank", "category", "type"]} for item in res] 
    except Exception as e:
        raise Exception (f"Cannot get OSM results ({osm_host}): {e}") 

In [420]:
get_osm(" Strokapelstraat, 14, 2235 Hulshout, Belgique")

[{'place_id': 1453627,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 335306395,
  'boundingbox': ['51.0396458', '51.0397282', '4.8105245', '4.8107318'],
  'lat': '51.03968045',
  'lon': '4.810627066701315',
  'display_name': '14, Strokapelstraat / Houtvensesteenweg, Houtvenne, Hulshout, Turnhout, Antwerpen, Vlaanderen, 2235, België / Belgique / Belgien',
  'place_rank': 30,
  'category': 'place',
  'type': 'house',
  'importance': 0.531,
  'address': {'house_number': '14',
   'road': 'Strokapelstraat / Houtvensesteenweg',
   'village': 'Houtvenne',
   'town': 'Hulshout',
   'county': 'Turnhout',
   'state': 'Antwerpen',
   'region': 'Vlaanderen',
   'postcode': '2235',
   'country': 'België / Belgique / Belgien',
   'country_code': 'be'},
  'namedetails': {}}]

In [172]:
addr.columns = pd.MultiIndex.from_tuples([("A", "addrKey"), ("A", "streetName"), ("B", "houseNumber"), ("B", "city"), ("B", "postCode"), ("C", "country"), ("test", "test")])
addr

A                                  B                             \
  addrKey             streetName houseNumber              city postCode   
0       0      Chaussée de Namur         198             LEUZE     5310   
1       1        Rue de la Halle          11             NAMUR     5000   
2       2  Avenue Jacques Sermon          38         GANSHOREN     1083   
3       3          rue du Centre         200            WAIMES     4950   
4       4     Kleine Waterstraat           9          HULSHOUT     2235   
5       5       rue Lambaitienne           7    NAMUR - WIERDE     5100   
6       6     Verschansingstraat           1     ANVERS-CENTRE     2000   
7       7       Rue des bouchers          39  BRUXELLES CENTRE     1000   
8       8        Beernaertstraat          27           BERCHEM     2600   
9       9          Place Jourdan          20         ETTERBEEK     1040   

          C test  
    country test  
0  Belgique    1  
1  Belgique    1  
2  Belgique    1  
3  Belgique    1  
4  Belgique    1  
5  Belgique    1  
6  Belgique    1  
7  Belgique    1  
8  Belgique    1  
9  Belgique    1

In [95]:
# addresses.iloc[0:10].T.to_dict()#groupby(["A", "B", "C"]).unstack().to_dict(orient="index")#.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict()).to_dict()
addr.columns.get_level_values(0).unique()

Index(['A', 'B', 'C'], dtype='object')

In [167]:
addr= addresses.iloc[0:10]
# [{l0: {rec for rec in addr[l0].to_dict(orient="records")} for l0 in addr.columns.get_level_values(0).unique()}]#
addr

L0   input                                                                \
0  addrKey             streetName houseNumber              city postCode   
0        0      Chaussée de Namur         198             LEUZE     5310   
1        1        Rue de la Halle          11             NAMUR     5000   
2        2  Avenue Jacques Sermon          38         GANSHOREN     1083   
3        3          rue du Centre         200            WAIMES     4950   
4        4     Kleine Waterstraat           9          HULSHOUT     2235   
5        5       rue Lambaitienne           7    NAMUR - WIERDE     5100   
6        6     Verschansingstraat           1     ANVERS-CENTRE     2000   
7        7       Rue des bouchers          39  BRUXELLES CENTRE     1000   
8        8        Beernaertstraat          27           BERCHEM     2600   
9        9          Place Jourdan          20         ETTERBEEK     1040   

L0           output  
0    country   test  
0   Belgique      1  
1   Belgique      1  
2   Belgique      1  
3   Belgique      1  
4   Belgique      1  
5   Belgique      1  
6   Belgique      1  
7   Belgique      1  
8   Belgique      1  
9   Belgique      1

In [104]:
l0='A'
[{lev0:rec for rec in addr[lev0].to_dict(orient="records")} for lev0 in addr.columns.get_level_values(0).unique()]

[{'A': {'addrKey': '9', 'streetName': 'Place Jourdan'}},
 {'B': {'houseNumber': '20', 'city': 'ETTERBEEK', 'postCode': '1040'}},
 {'C': {'country': 'Belgique'}}]

In [176]:
# [rec for rec in addr.to_dict(orient="dict")]
addr#.to_dict(orient="index")
# addr = addr.copy()

A                                  B                             \
  addrKey             streetName houseNumber              city postCode   
0       0      Chaussée de Namur         198             LEUZE     5310   
1       1        Rue de la Halle          11             NAMUR     5000   
2       2  Avenue Jacques Sermon          38         GANSHOREN     1083   
3       3          rue du Centre         200            WAIMES     4950   
4       4     Kleine Waterstraat           9          HULSHOUT     2235   
5       5       rue Lambaitienne           7    NAMUR - WIERDE     5100   
6       6     Verschansingstraat           1     ANVERS-CENTRE     2000   
7       7       Rue des bouchers          39  BRUXELLES CENTRE     1000   
8       8        Beernaertstraat        None           BERCHEM     2600   
9       9          Place Jourdan          20         ETTERBEEK     1040   

          C test  
    country test  
0  Belgique    1  
1  Belgique    1  
2  Belgique    1  
3  Belgique    1  
4  Belgique    1  
5  Belgique    1  
6  Belgique    1  
7  Belgique    1  
8  Belgique    1  
9  Belgique    1

In [180]:
addr#.loc[8, ('B', 'houseNumber')] = None

A                                  B                             \
  addrKey             streetName houseNumber              city postCode   
0       0      Chaussée de Namur         198             LEUZE     5310   
1       1        Rue de la Halle          11             NAMUR     5000   
2       2  Avenue Jacques Sermon          38         GANSHOREN     1083   
3       3          rue du Centre         200            WAIMES     4950   
4       4     Kleine Waterstraat           9          HULSHOUT     2235   
5       5       rue Lambaitienne           7    NAMUR - WIERDE     5100   
6       6     Verschansingstraat           1     ANVERS-CENTRE     2000   
7       7       Rue des bouchers          39  BRUXELLES CENTRE     1000   
8       8        Beernaertstraat        None           BERCHEM     2600   
9       9          Place Jourdan          20         ETTERBEEK     1040   

          C test  
    country test  
0  Belgique    1  
1  Belgique    1  
2  Belgique    1  
3  Belgique    1  
4  Belgique    1  
5  Belgique    1  
6  Belgique    1  
7  Belgique    1  
8  Belgique    1  
9  Belgique    1

In [184]:
res = [{ k1: {k2: rec[(k1, k2)]  for k2 in addr[k1].columns.get_level_values(0)} for k1 in addr.columns.get_level_values(0).unique() }  for rec in addr.to_dict(orient="records")]

In [187]:
[{k1: {k2: rec[k1][k2] for k2 in rec[k1] if not pd.isnull(rec[k1][k2]) and rec[k1][k2] != ""} for k1 in rec} for rec in res]

[{'A': {'addrKey': '0', 'streetName': 'Chaussée de Namur'},
  'B': {'houseNumber': '198', 'city': 'LEUZE', 'postCode': '5310'},
  'C': {'country': 'Belgique'},
  'test': {'test': 1}},
 {'A': {'addrKey': '1', 'streetName': 'Rue de la Halle'},
  'B': {'houseNumber': '11', 'city': 'NAMUR', 'postCode': '5000'},
  'C': {'country': 'Belgique'},
  'test': {'test': 1}},
 {'A': {'addrKey': '2', 'streetName': 'Avenue Jacques Sermon'},
  'B': {'houseNumber': '38', 'city': 'GANSHOREN', 'postCode': '1083'},
  'C': {'country': 'Belgique'},
  'test': {'test': 1}},
 {'A': {'addrKey': '3', 'streetName': 'rue du Centre'},
  'B': {'houseNumber': '200', 'city': 'WAIMES', 'postCode': '4950'},
  'C': {'country': 'Belgique'},
  'test': {'test': 1}},
 {'A': {'addrKey': '4', 'streetName': 'Kleine Waterstraat'},
  'B': {'houseNumber': '9', 'city': 'HULSHOUT', 'postCode': '2235'},
  'C': {'country': 'Belgique'},
  'test': {'test': 1}},
 {'A': {'addrKey': '5', 'streetName': 'rue Lambaitienne'},
  'B': {'houseNumb

In [136]:
addresses[("input",)]

,addrKey,streetName,houseNumber,city,postCode,country
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique
1,1,Rue de la Halle,11,NAMUR,5000,Belgique
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique
3,3,rue du Centre,200,WAIMES,4950,Belgique
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique
...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique


In [139]:
addresses.columns

MultiIndex([('input',     'addrKey'),
            ('input',  'streetName'),
            ('input', 'houseNumber'),
            ('input',        'city'),
            ('input',    'postCode'),
            ('input',     'country')],
           names=['L0', 0])

In [144]:
addresses[("output", "test")]=1

In [145]:
addresses

L0     input                                                                  \
0    addrKey                streetName houseNumber             city postCode   
0          0         Chaussée de Namur         198            LEUZE     5310   
1          1           Rue de la Halle          11            NAMUR     5000   
2          2     Avenue Jacques Sermon          38        GANSHOREN     1083   
3          3             rue du Centre         200           WAIMES     4950   
4          4        Kleine Waterstraat           9         HULSHOUT     2235   
...      ...                       ...         ...              ...      ...   
2995    2995       Rue du Wainage(VEL)          62      Sambreville     5060   
2996    2996  Chaussée de Tournai(R-C)           4          Tournai     7520   
2997    2997         Rue Louis Caty(B)          32   Saint-Ghislain     7331   
2998    2998        Drève des Bouleaux           5  Braine-le-Comte     7090   
2999    2999                Rue Rogier           1              Spa     4900   

L0             output  
0      country   test  
0     Belgique      1  
1     Belgique      1  
2     Belgique      1  
3     Belgique      1  
4     Belgique      1  
...        ...    ...  
2995  Belgique      1  
2996  Belgique      1  
2997  Belgique      1  
2998  Belgique      1  
2999  Belgique      1  

[3000 rows x 7 columns]